In [1]:
!pip install git+https://github.com/esgantivar/qmc.git -q

     |████████████████████████████████| 368kB 18.7MB/s 


In [2]:
import numpy as np
import tensorflow as tf
from qmc.tf.tt.models import MPSClassifierQMC
from qmc.tf.tt.utils import ReturnBestEarlyStopping
from qmc.tf.tt import features

/usr/local/lib/python3.7/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking qmc.tf.layers.CrossProduct.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.7/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking qmc.tf.layers.DensityMatrix2Dist.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))
/usr/local/lib/python3.7/dist-packages/typeguard/__init__.py:804: UserWarning: no type annotations present -- not typechecking qmc.tf.layers.DensityMatrixRegression.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


In [3]:
#Extract the MNIST Images and preprocess
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# convert class vectors to binary class matrices
y_train_categorical = tf.keras.utils.to_categorical(y_train, 10)
y_test_categorical = tf.keras.utils.to_categorical(y_test, 10)
# Y one hot
Y_train = y_train[:, np.newaxis]
Y_test = y_test[:, np.newaxis]

# convert images to supported format
x_train_feature = features.binary_map(x_train)
x_test_feature = features.binary_map(x_test)
x_train.shape, y_train.shape, x_test.shape, y_train.shape, y_train_categorical.shape, y_test_categorical.shape, Y_train.shape, Y_test.shape, x_train_feature.shape, x_test_feature.shape

11493376/11490434 [==============================] - 0s 0us/step


((60000, 28, 28),
 (60000,),
 (10000, 28, 28),
 (60000,),
 (60000, 10),
 (10000, 10),
 (60000, 1),
 (10000, 1),
 (60000, 784, 2),
 (10000, 784, 2))

In [4]:
num_exps = 5
lr = 0.0001
batch_size = 256
epochs = 50
verbose = 0
patience = 10
bonds = [2, 4, 8, 16]
results = np.zeros((len(bonds), num_exps))

In [5]:
for d_i in range(len(bonds)):
  d_bond = bonds[d_i]
  for i in range(num_exps):
    model = MPSClassifierQMC(num_eig=10, d_bond=d_bond, n_output=10)
    earlyStopping = ReturnBestEarlyStopping(monitor='val_accuracy', patience=patience, verbose=1, mode='max', restore_best_weights=True)
    callback_list = [earlyStopping]
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                  optimizer=optimizer,
                  metrics=['accuracy'])
    history = model.fit(x_train_feature, y_train_categorical,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=verbose,
                        callbacks=callback_list,
                        validation_data=(x_test_feature, y_test_categorical))
    out = model.predict(x_test_feature, batch_size=batch_size)
    out = np.argmax(out, axis=1)
    nt = len(Y_test)
    errors_mixed_qmkdc = np.sum(out != Y_test.reshape((nt)))
    accuracy_mixed_qmkdc = 1 - errors_mixed_qmkdc/nt
    print(f'd_bond: {d_bond}, i: {i}, accuracy: {round(accuracy_mixed_qmkdc, 4)}')
    results[d_i][i] = accuracy_mixed_qmkdc
  print(results[d_i])
results

Restoring model weights from the end of the best epoch.

Epoch 15: early stopping
d_bond: 2, i: 0, accuracy: 0.6626
Restoring model weights from the end of the best epoch.

Epoch 11: early stopping
d_bond: 2, i: 1, accuracy: 0.098
Restoring model weights from the end of the best epoch.

Epoch 13: early stopping
d_bond: 2, i: 2, accuracy: 0.759
Restoring model weights from the end of the best epoch.

Epoch 11: early stopping
d_bond: 2, i: 3, accuracy: 0.098
Restoring model weights from the end of the best epoch.

Epoch 11: early stopping
d_bond: 2, i: 4, accuracy: 0.5612
[0.6626 0.098  0.759  0.098  0.5612]
Restoring model weights from the end of the best epoch.

Epoch 12: early stopping
d_bond: 4, i: 0, accuracy: 0.7956
Restoring model weights from the end of the best epoch.

Epoch 11: early stopping
d_bond: 4, i: 1, accuracy: 0.098
Restoring model weights from the end of the best epoch.

Epoch 13: early stopping
d_bond: 4, i: 2, accuracy: 0.8672
Restoring model weights from the end of

array([[0.6626, 0.098 , 0.759 , 0.098 , 0.5612],
       [0.7956, 0.098 , 0.8672, 0.098 , 0.8918],
       [0.8152, 0.9444, 0.098 , 0.931 , 0.9333],
       [0.9732, 0.911 , 0.9448, 0.9739, 0.9697]])